# Preparation
<a id=preparation />

In [ ]:
%run "../Functions/2. Google form analysis.ipynb"

[Samples](#Samples)

# April 2018 Playtest

In [ ]:
#rmdf1522.columns
#'coordinates', 'customData', 'customData.biobrick', 'customData.chapter', 'customData.controls', 'customData.count', 'customData.device', 'customData.devices', 'customData.dnabit', 'customData.duration', 'customData.energy', 'customData.graphics', 'customData.language', 'customData.life', 'userId', 'customData.message', 'customData.nanobot', 'customData.newtab', 'customData.option', 'customData.plasmid', 'customData.platform', 'customData.sametab', 'customData.slot', 'customData.sound', 'customData.source', 'customData.total', 'id', 'playerCustomData', 'sessionId', 'section', 'serverTime', 'type', 'userTime'

In [ ]:
#testUserId1 = 'f4589007-0809-4673-ad50-d985abe4fa65'
#testUserId2 = '653f174e-c23f-4d3a-94f6-d26b53ac7168'
#gform[gform[QUserId] == testUserId2]
#testUserId2 in testUsers

In [ ]:
displayResults = False

## labels

In [ ]:
# rows
phase1Label = 'phase1'
phase2Label = 'phase2'
totalLabel = 'total'

# columns
playersCountLabel = 'players'
respondentsCountLabel = 'respondents'
respondentsExploitableCountLabel = 'exploitable respondents'
respondentsTwiceCountLabel = 'twice respondents'
respondentsTwiceVolunteersCountLabel = 'volunteers'
prefixFemale = 'f '
prefixMale = 'm '


playtestStatistics = pd.DataFrame(
    data = np.nan,
    index = [phase1Label, phase2Label, totalLabel],
    columns = [playersCountLabel]
)

## functions

In [ ]:
def getFormattedPercentageAndAbsoluteValueFunction(total):
    return lambda p: '{:1.1f}%'.format(p) + ' ({:.0f})'.format(round(p * total / 100))

In [ ]:
# problem: counts UserIds, but there can be more than one subject per UserId
def setCount(sample, rowLabel, columnLabel, useProfiles=True):
    if useProfiles:
        playtestStatistics.loc[rowLabel, columnLabel] = len(getUniqueProfiles(sample))
    else:
        playtestStatistics.loc[rowLabel, columnLabel] = len(sample[QUserId].unique())
    return playtestStatistics.loc[rowLabel, columnLabel]

def setGenderedCount(sampledGform, rowLabel, columnLabel, plotPie = False, title = ''):
    femaleSample = sampledGform[sampledGform[QGender] == "Female"]
    maleSample = sampledGform[sampledGform[QGender] == "Male"]

    tCount = setCount(sampledGform, rowLabel, columnLabel)
    fCount = setCount(femaleSample, rowLabel, prefixFemale + columnLabel)
    mCount = setCount(maleSample, rowLabel, prefixMale + columnLabel)
    oCount = tCount-fCount-mCount
    
    if plotPie:
        plt.pie([fCount, mCount, oCount], labels=['Female', 'Male', 'Other/Unspecified/Unknown'],\
                autopct=getFormattedPercentageAndAbsoluteValueFunction(tCount),\
                shadow=True)
        plt.title(title)
        plt.show()

In [ ]:
def getUniqueProfiles(sampleForm):
    playtestProfiles = sampleForm.loc[:,[QUserId, QAge, QGender]]
    playtestProfiles.drop_duplicates(subset=[QUserId, QAge, QGender], keep='first', inplace=True)
    return playtestProfiles

In [ ]:
# samples using UserIds associated with unique profiles
def getUserIdsWithUniqueProfile(sampleForm):
    multipleProfiles = sampleForm.loc[:, [QUserId, QAge, QGender]].drop_duplicates()[QUserId].value_counts()
    multipleProfiles = multipleProfiles[multipleProfiles != 1]    
    return sampleForm[~sampleForm[QUserId].isin(multipleProfiles.index)]

In [ ]:
#multipleProfiles = samplePlaytest.loc[:, [QUserId, QAge, QGender]].drop_duplicates()[QUserId].value_counts()
#multipleProfiles = multipleProfiles[multipleProfiles != 1]
#for userId in multipleProfiles.index:
#    print(str(samplePlaytest[samplePlaytest[QUserId] == userId].loc[:, [QUserId, QAge, QGender]]))

In [ ]:
def getTicksByStep(minValue, maxValue, step):
    start = minValue + step - (minValue % step)
    end = maxValue - (maxValue % step) + step

    ticks = [x for x in range(start, end, step)]
    ticks = np.append([minValue], ticks)
    ticks = np.append(ticks, [maxValue])

    return ticks

In [ ]:
def getTicks(minValue, maxValue, step = 0, labelCount = 0):
    if step == 0 and labelCount != 0:
        step = int((maxValue - minValue + 1) / labelCount)
    return getTicksByStep(minValue, maxValue, step)

In [ ]:
def plotAge(sampleForm, title = 'Âge des participants', xlabel = 'Âge', ylabel = 'Nombre'):
    playtestProfiles = getUniqueProfiles(sampleForm)

    ages = playtestProfiles[QAge].value_counts().sort_index()
    minAge = ages.index.min()
    maxAge = ages.index.max()
    ages = pd.Series(ages, index=range(minAge, maxAge+1)).fillna(0)
    
    yticks = getTicks(int(ages.min()), int(ages.max()), labelCount = 5)
    ax = ages.plot(kind='bar', yticks=yticks)
    
    #ax = ages.plot(kind='bar')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    # xticks like yticks doesn't work
    # workaround
    xticks = getTicks(minAge, maxAge, step = 5)
    newTicks = xticks - minAge
    newLabels = xticks
    ax.xaxis.set_ticks(newTicks)
    ax.xaxis.set_ticklabels(newLabels)
    
    plt.title(title)
    plt.show()

In [ ]:
#sampleForm = samplePlaytest.copy()
#pivotalAgesList = [3,15,25,35,45,55]
#title = 'Âge des participants'

def plotAgeGroups(sampleForm, pivotalAgesList = [], title = 'Âge des participants',\
                  showCount = False, unknownAgeLabel = 'unknown'):
    playtestProfiles = getUniqueProfiles(sampleForm)

    ages = playtestProfiles[QAge]
    maxAge = ages.max()
    minAge = ages.min()
    otherAge = maxAge + 1

    pivotalAgesList.sort()
    pivotalAgesList = [x for x in pivotalAgesList if (x < maxAge and x > minAge)]
    pivotalAgesList = np.append(pivotalAgesList, [maxAge, otherAge])

    ageBins = pd.Series(0,index=pivotalAgesList)
    labels = pd.Series('+++',index=pivotalAgesList)
    previous = str(minAge)

    for pivot in pivotalAgesList:
        if pivot != otherAge:
            ageBins[pivot] = len(ages[ages <= pivot])
            labels[pivot] = previous + '-' + str(pivot)
            if showCount:
                labels[pivot] += " (" + str(ageBins[pivot]) + ")"
            ages = ages[ages > pivot]
            previous = str(pivot)
        else:
            ages = playtestProfiles[QAge]
            ageBins[otherAge] = len(ages[pd.isnull(ages)])
            labels[otherAge] = unknownAgeLabel
            if showCount:
                labels[otherAge] += " (" + str(ageBins[otherAge]) + ")"

    plt.pie(ageBins.values, labels=labels,\
            #autopct='%1.1f%%',\
            autopct=getFormattedPercentageAndAbsoluteValueFunction(len(playtestProfiles)),\
            shadow=True)
    plt.title(title)
    plt.show()

In [ ]:
def plotGender(sampleForm, title = '', labels = ['Femmes', 'Hommes', 'Autres/Non précisé']):
    playtestProfiles = getUniqueProfiles(sampleForm)

    fCount = len(playtestProfiles[playtestProfiles[QGender] == 'Female'])
    mCount = len(playtestProfiles[playtestProfiles[QGender] == 'Male'])
    tCount = len(playtestProfiles)
    oCount = tCount-fCount-mCount

    plt.pie([fCount, mCount, oCount], labels=labels,\
            autopct=getFormattedPercentageAndAbsoluteValueFunction(tCount),\
            shadow=True)
    plt.title(title)
    plt.show()
    print("t="+str(tCount))
    print("f="+str(fCount))
    print("m="+str(mCount))
    print("o="+str(oCount))

In [ ]:
def getDuplicates(sample, columns):
    sampled = sample.loc[:,columns]
    return sampled,sampled[sampled.duplicated()]

In [ ]:
#sample = samplePlaytest.copy()
#columns = [QUserId, QAge, QGender, QTemporality]
def getTimedAssembledDuplicates(sample, columns, displayColumns = []):
    if len(displayColumns) == 0:
        displayColumns = columns
        
    sampled,duplicates = getDuplicates(sample, columns)
    dupIndex = pd.Index([])
    for dup in duplicates.values:
        dupbool = [(x == dup).all() for x in sampled.values]
        dupIndex = dupIndex.union(sampled.index[dupbool])
    result = sample.loc[dupIndex, displayColumns].sort_values(by = displayColumns)
    return result

In [ ]:
# handpicked error cases
pretestPosttestPairsErrorCases = [
  # 2 posttests
    "4b19e51b-c81f-4cbb-af72-0dced3bbb52b",
    "1e6cb7d9-56f5-48a5-9bb8-14885325add6",
    "9562d4e4-3d36-4399-a95f-7784328c2683",

  # 1 posttest
    "e79d806e-3d0b-49f8-a5e8-23de5548b770",
    "8d1f1ec8-379e-4ec6-88ba-2fdc1d0989be",

  # 1 pretest
    "3006900c-37d2-44dd-a8e6-674f6fd1cae3",
    "721cb935-f939-40ae-b56f-01d97ea23604",

  # 2 posttests, 1 pretest
    "38ef3b8d-dedc-4224-be1c-5d7372973cc0",
    "96470842-aa9a-4802-9e23-38060a1980b4",

  # 2 pretests
    "f8920e82-1cdf-43a0-aedb-06ddaf9960d6",
    "86c71536-ee24-4c3d-b856-6a97b63f6217",

  # 1 pretest, 1 posttest on 2 different days
    "4cdb6030-4547-4c8f-9d9e-286cff786157",
    "9d09b624-c719-446b-a738-e7059dd1a8f4",
]
#incorrectPairsSurveys = samplePlaytestUniqueProfiles[samplePlaytestUniqueProfiles[QUserId].isin(pretestPosttestPairsErrorCases)]
#len(incorrectPairsSurveys)

In [ ]:
#_gformDF = incorrectPairsSurveys.copy()
#_gformDF = gform.copy()

def getPerfectPretestPostestPairs(_gformDF):
    #counter = 0
    #userIdsToKeep = _gformDF[QUserId].unique()
    #print("step" + str(counter) + ": " + str(len(userIdsToKeep)))
    #counter+=1

    # remove incomplete data
    #  consequently removes posttests without pretests
    _gformDF2 = getWithoutIncompleteAnswers(_gformDF)
    #print("step" + str(counter) + ": " + str(len(_gformDF2[QUserId].unique())))
    #counter+=1

    # keep userIds with exactly 1 pretest and posttest
    #userIdsToKeep = []
    valueCounts = _gformDF2[_gformDF2[QTemporality] == answerTemporalities[0]][QUserId].value_counts()
    _gformDF3 = _gformDF2[_gformDF2[QUserId].isin(valueCounts[valueCounts == 1].index.values)]
    #print("step" + str(counter) + ": " + str(len(_gformDF3[QUserId].unique())))
    #counter+=1

    valueCounts = _gformDF3[_gformDF3[QTemporality] == answerTemporalities[1]][QUserId].value_counts()
    _gformDF4 = _gformDF3[_gformDF3[QUserId].isin(valueCounts[valueCounts == 1].index.values)]
    #print("step" + str(counter) + ": " + str(len(_gformDF4[QUserId].unique())))
    #counter+=1
    
    def getFloorDayFromUserIdAndTemporality(userId,i,df):
        return df[(df[QUserId] == userId)\
              &(df[QTemporality] == answerTemporalities[i])]\
            .iloc[0][QTimestamp].floor('d')

    # only keep those with pretest-posttest pairs on the same day
    userIds = _gformDF4[QUserId].unique()
    userIdsDF = pd.DataFrame(pd.NaT, index=userIds, columns=["0","1"])
    for i in [0, 1]:
        userIdsDF[str(i)] = pd.Series([getFloorDayFromUserIdAndTemporality(x,i,_gformDF4) for x in userIds], index = userIds)
    userIds = userIdsDF.index[userIdsDF.loc[:,"0"] == userIdsDF.loc[:,"1"]]
    _gformDF5 = _gformDF4[_gformDF4[QUserId].isin(userIds)]
    #print("step" + str(counter) + ": " + str(len(_gformDF5[QUserId].unique())))
    #counter+=1

    return _gformDF5

In [ ]:
def getVolunteers(sample):
    volunteerUserIds = sample[sample[QVolunteer].isin(yesNoPositives)][QUserId]
    return sample[sample[QUserId].isin(volunteerUserIds)]

## Dates

In [ ]:
playtest2018Phase1Days = [
    pd.Timestamp('2018-04-10', tz='utc'),	# Tuesday
    pd.Timestamp('2018-04-11', tz='utc'),	# Wednesday

    pd.Timestamp('2018-04-13', tz='utc'),	# Friday
    pd.Timestamp('2018-04-14', tz='utc'),	# Saturday


    pd.Timestamp('2018-04-17', tz='utc'),	# Tuesday
    pd.Timestamp('2018-04-18', tz='utc'),	# Wednesday

    pd.Timestamp('2018-04-20', tz='utc'),	# Friday
    pd.Timestamp('2018-04-21', tz='utc'),	# Saturday


    pd.Timestamp('2018-04-24', tz='utc'),	# Tuesday
    pd.Timestamp('2018-04-25', tz='utc'),	# Wednesday
]

playtest2018Phase2Days = [
    pd.Timestamp('2018-04-27', tz='utc'),	# Friday
    pd.Timestamp('2018-04-28', tz='utc'),	# Saturday
]

playtest2018Days = np.append(playtest2018Phase1Days, playtest2018Phase2Days).tolist()

In [ ]:
playtest2018EuBerTimeStartTime = datetime.time(11, 0)
playtest2018EuBerTimeEndTime = datetime.time(19, 0)

### aliases for experiment boundaries

In [ ]:
playtest2018Phase1StartDate = pd.Timestamp.combine(min(playtest2018Phase1Days).date(), playtest2018EuBerTimeStartTime).tz_localize('Europe/Berlin')
playtest2018Phase1EndDate   = pd.Timestamp.combine(max(playtest2018Phase1Days).date(), playtest2018EuBerTimeEndTime).tz_localize('Europe/Berlin')
playtest2018Phase2StartDate = pd.Timestamp.combine(min(playtest2018Phase2Days).date(), playtest2018EuBerTimeStartTime).tz_localize('Europe/Berlin')
playtest2018Phase2EndDate   = pd.Timestamp.combine(max(playtest2018Phase2Days).date(), playtest2018EuBerTimeEndTime).tz_localize('Europe/Berlin')
playtest2018Phase1StartDate,playtest2018Phase1EndDate,playtest2018Phase2StartDate,playtest2018Phase2EndDate

In [ ]:
# warning: timestamp = pd.Timestamp('2018-04-24 11:00:11.381000+0000', tz='Europe/Berlin')
#  actually yields Timestamp('2018-04-24 13:00:11.381000+0200', tz='Europe/Berlin') because
#  '+....' codes for timezone, eg here '+0000' codes for UTC timezone

def isPlaytest(timestamp, days = playtest2018Days):
    euBerTimestamp = timestamp.tz_convert('Europe/Berlin')
    euBerTimestampTime = euBerTimestamp.time()
    result = (timestamp.tz_convert('utc').floor('d') in days)\
    and (euBerTimestampTime >= playtest2018EuBerTimeStartTime)\
    and (playtest2018EuBerTimeEndTime >= euBerTimestampTime)
    return result

def isPlaytestPhase1(timestamp):
    return isPlaytest(timestamp, days = playtest2018Phase1Days)

def isPlaytestPhase2(timestamp):
    return isPlaytest(timestamp, days = playtest2018Phase2Days)

#### isPlaytest day and hour test

In [ ]:
assert(~any([isPlaytest(x) for x in playtest2018Phase1Days]))

In [ ]:
testCorrectTimes = [
                pd.Timestamp('11:00:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('12:43:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('14:58:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('16:38:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('19:00:00', tz='Europe/Berlin').tz_convert('utc'),
]
testIncorrectTimes = [
                pd.Timestamp('19:01:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('21:43:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('23:13:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('06:41:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('10:59:00', tz='Europe/Berlin').tz_convert('utc'),
]

for time in testCorrectTimes:
    correctPlaytestTimes = [pd.Timestamp.combine(x.date(), time.time()).tz_localize('utc') for x in playtest2018Phase1Days]
    assert(all([isPlaytest(x) for x in correctPlaytestTimes]))
    
for time in testIncorrectTimes:
    correctPlaytestTimes = [pd.Timestamp.combine(x.date(), time.time()).tz_localize('utc') for x in playtest2018Phase1Days]
    assert(~any([isPlaytest(x) for x in correctPlaytestTimes]))

## Samples
samples based on dates and platform

In [ ]:
sampleWhole = getWithoutTest(gform)

# rmdfConcat is rmdf1522 + rmdf160
rmdfLinuxPlatform = rmdfConcat[rmdfConcat['customData.platform'] == '"linuxplayer"']
rmdfLinuxPlatform = rmdfConcat[rmdfConcat[QUserId].isin(rmdfLinuxPlatform[QUserId])]

# sample by platform using RedMetrics
sampleLinux = getWithoutTest(sampleWhole[sampleWhole[QUserId].isin(rmdfLinuxPlatform[QUserId].unique())])

# sample by time slots
samplePlaytestPhase1 = sampleLinux[sampleLinux[QTimestamp].apply(isPlaytestPhase1)]
samplePlaytestPhase2 = sampleLinux[sampleLinux[QTimestamp].apply(isPlaytestPhase2)]
samplePlaytest = pd.concat([samplePlaytestPhase1, samplePlaytestPhase2])

# sample by filtering out userIds with multiple profiles
#  better solution: take those multiple profiles into account, eg by creating new userIds for them
samplePlaytestUniqueProfilesPhase1 = getUserIdsWithUniqueProfile(samplePlaytestPhase1)
samplePlaytestUniqueProfilesPhase2 = getUserIdsWithUniqueProfile(samplePlaytestPhase2)
samplePlaytestUniqueProfiles       = getUserIdsWithUniqueProfile(samplePlaytest)

# sample by keeping only pretest-posttest pairs
samplePlaytestPretestPosttestUniqueProfilesPhase1 = getPerfectPretestPostestPairs(samplePlaytestUniqueProfilesPhase1)
samplePlaytestPretestPosttestUniqueProfilesPhase2 = getPerfectPretestPostestPairs(samplePlaytestUniqueProfilesPhase2)
samplePlaytestPretestPosttestUniqueProfiles       = getPerfectPretestPostestPairs(samplePlaytestUniqueProfiles)

# sample according to the answer to the Volunteering question
samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase1 = getVolunteers(samplePlaytestPretestPosttestUniqueProfilesPhase1)
samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase2 = getVolunteers(samplePlaytestPretestPosttestUniqueProfilesPhase2)
samplePlaytestPretestPosttestUniqueProfilesVolunteers       = getVolunteers(samplePlaytestPretestPosttestUniqueProfiles)

In [ ]:
if displayResults:
#if True:
    print("sampleWhole="+str(len(sampleWhole)))
    print("sampleLinux="+str(len(sampleLinux)))
    print("samplePlaytestPhase1="+str(len(samplePlaytestPhase1)))
    print("samplePlaytestPhase2="+str(len(samplePlaytestPhase2)))
    print("samplePlaytest="+str(len(samplePlaytest)))
    print("samplePlaytestUniqueProfilesPhase1="+str(len(samplePlaytestUniqueProfilesPhase1)))
    print("samplePlaytestUniqueProfilesPhase2="+str(len(samplePlaytestUniqueProfilesPhase2)))
    print("samplePlaytestUniqueProfiles="+str(len(samplePlaytestUniqueProfiles)))
    print("samplePlaytestPretestPosttestUniqueProfilesPhase1="+str(len(samplePlaytestPretestPosttestUniqueProfilesPhase1)))
    print("samplePlaytestPretestPosttestUniqueProfilesPhase2="+str(len(samplePlaytestPretestPosttestUniqueProfilesPhase2)))
    print("samplePlaytestPretestPosttestUniqueProfiles="+str(len(samplePlaytestPretestPosttestUniqueProfiles)))
    print("samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase1="+str(len(samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase1)))
    print("samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase2="+str(len(samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase2)))
    print("samplePlaytestPretestPosttestUniqueProfilesVolunteers="+str(len(samplePlaytestPretestPosttestUniqueProfilesVolunteers)))
    
    print()
    print("unique userIds sampleWhole=" + str(sampleWhole[QUserId].nunique()))
    print("unique userIds sampleLinux=" + str(sampleLinux[QUserId].nunique()))
    print("unique userIds samplePlaytestPhase1=" + str(samplePlaytestPhase1[QUserId].nunique()))
    print("unique userIds samplePlaytestPhase2=" + str(samplePlaytestPhase2[QUserId].nunique()))
    print("unique userIds samplePlaytest=" + str(samplePlaytest[QUserId].nunique()))
    print("unique userIds samplePlaytestUniqueProfilesPhase1=" + str(samplePlaytestUniqueProfilesPhase1[QUserId].nunique()))
    print("unique userIds samplePlaytestUniqueProfilesPhase2=" + str(samplePlaytestUniqueProfilesPhase2[QUserId].nunique()))
    print("unique userIds samplePlaytestUniqueProfiles=" + str(samplePlaytestUniqueProfiles[QUserId].nunique()))
    print("unique userIds samplePlaytestPretestPosttestUniqueProfilesPhase1=" + str(samplePlaytestPretestPosttestUniqueProfilesPhase1[QUserId].nunique()))
    print("unique userIds samplePlaytestPretestPosttestUniqueProfilesPhase2=" + str(samplePlaytestPretestPosttestUniqueProfilesPhase2[QUserId].nunique()))
    print("unique userIds samplePlaytestPretestPosttestUniqueProfiles=" + str(samplePlaytestPretestPosttestUniqueProfiles[QUserId].nunique()))
    print("unique userIds samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase1="+str(samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase1[QUserId].nunique()))
    print("unique userIds samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase2="+str(samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase2[QUserId].nunique()))
    print("unique userIds samplePlaytestPretestPosttestUniqueProfilesVolunteers="+str(samplePlaytestPretestPosttestUniqueProfilesVolunteers[QUserId].nunique()))

#### Checks on the profiles counts - search for duplicates

In [ ]:
if False:
    # duplicate profiles
    #  may be people who filled in the same survey twice in a row
    #  or different people who happened to belong to same demographic group and play on same unreset computer
    #  Only way to differentiate: look at RedMetrics events
    columns = [QUserId, QAge, QGender, QTemporality]
    displayColumns = [QUserId, QAge, QGender, QTemporality, QTimestamp]
    duplicates = getTimedAssembledDuplicates(samplePlaytest, columns, displayColumns)
    grouped = duplicates.groupby(columns)
    grouped.size()

In [ ]:
if False:
    # duplicates per day
    duplicatesPerDay = duplicates.copy()
    duplicatesPerDay.loc[:, QTimestamp] = duplicatesPerDay.loc[:, QTimestamp].apply(pd.Timestamp.date)
    duplicatesPerDay.groupby(QTimestamp).size()

In [ ]:
if False:
    # duplicates or just unpaired surveys? (ie pretest wihtout posttest or conversely)
    #  timely repartition of redmetrics events is inconclusive: analysis of RedMetrics events is necessary
    duplicates['diff'] = duplicates[QTimestamp].diff()
    duplicates

In [ ]:
#userId = duplicates.loc[666, QUserId]
#n = 0
#userId = "29b739fc-4f9f-4f5e-bfee-8ba12de4b7fa"
#rmdf = rmdfConcat
#surveys = gform.copy()

def plotEventsPerMinute(userId, rmdf, surveys = [], n = 0):
    print("\n\nuserId="+str(userId))
    times = rmdf[rmdf[QUserId] == userId]['userTime']
    times.index = range(len(times))
    days = times.dt.date
    # compute day of experiment
    ratios = [len(days[days == d])/len(days) for d in days.unique()]
    ratioratios = pd.Series([r/max(ratios) for r in ratios], index = days.unique())
    importantDays = days.unique()[ratioratios > 0.05]
    if len(importantDays) > 1:
        print("more than one important day!")
    times = times.dt.floor('min')
    times = times[times.dt.date.isin(importantDays)]
    startGraph = times.min()
    endGraph = times.max()
    print("\nevents:\nfirst:" + str(startGraph) + "\nlast:" + str(endGraph))
    minutesCount = int((endGraph - startGraph).total_seconds()/60)

    fig = plt.figure()
    ax = fig.add_subplot(111)

    xrot = 45
    alpha = 0.5

    if(len(surveys) > 0):
        surveySubmissions = surveys[surveys[QUserId] == userId]
        surveySubmissionsTimes = surveySubmissions[QTimestamp].dt.floor('min')
        startGraph = min(startGraph, surveySubmissionsTimes.min())
        endGraph = max(endGraph, surveySubmissionsTimes.max())
        minutesCount = int((endGraph - startGraph).total_seconds()/60)

        pretestSubmissions = surveySubmissions[surveySubmissions[QTemporality] == answerTemporalities[0]][QTimestamp].dt.floor('min')
        posttestSubmissions = surveySubmissions[surveySubmissions[QTemporality] == answerTemporalities[1]][QTimestamp].dt.floor('min')

        ax2 = ax.twinx()
        ax3 = ax.twinx()
        
        ax2.set_yticks([])
        ax3.set_yticks([])
        ax2.set_yticklabels([])
        ax3.set_yticklabels([])

    plotRange = (startGraph,endGraph)
        
    times.hist(bins=minutesCount, ax = ax, xrot = xrot, color='blue', alpha=alpha, range=plotRange, grid=False)

    if(len(surveys) > 0):

        if len(pretestSubmissions) > 0:
            print("\npretests:\n" + str(pretestSubmissions))
            pretestSubmissions[n:].hist(bins = 20, ax = ax2, xrot = xrot, color='green', alpha=alpha, range=plotRange, grid=False)

        if len(posttestSubmissions) > 0:
            print("\nposttests:\n" + str(posttestSubmissions))
            posttestSubmissions.hist(bins = 20, ax = ax3, xrot = xrot, color='red', alpha=alpha, range=plotRange, grid=False)

    plt.show()

In [ ]:
def getResetConfigurationEvents(rmdf):
    rmdf[(rmdf['type'] == 'selectmenu') & (rmdf['customData.option'] == '"RESETCONFIGURATION"')].loc[:, [QUserId, 'userTime', 'customData.option']]

In [ ]:
def getProfileCounts(sampleForm, n = 0):
    sampled = sampleForm.loc[:,[QUserId, QGender, QAge]].drop_duplicates()[QUserId].value_counts().sort_values(ascending=False)
    return sampled[sampled >= n]

In [ ]:
if False:
    plotEventsPerMinute("e3fc1686-6802-4e36-9877-d1bb226a6f72", rmdfConcat, samplePlaytest)

In [ ]:
if False:
    gform[gform[QUserId] == "29b739fc-4f9f-4f5e-bfee-8ba12de4b7fa"].loc[:, [QUserId, QAge, QGender, QTemporality, QTimestamp]]

In [ ]:
if False:
    for userId in duplicates[QUserId].unique():
        print(str(samplePlaytest[samplePlaytest[QUserId] == userId].loc[:,displayColumns]))
        plotEventsPerMinute(userId, rmdfConcat, gform)

In [ ]:
if False:
    getTimedAssembledDuplicates(samplePlaytest, [QUserId, QAge, QGender], [QUserId, QAge, QGender, QTemporality, QTimestamp])

------------

# Global analysis of player paths

In [ ]:
#userIds = _gformDF4[QUserId].unique()
#userIds = pd.DataFrame(pd.NaT, index=userIds, columns=["0","1"])
#for i in [0, 1]:
#    userIds[str(i)] = pd.Series([getFloorDayFromUserIdAndTemporality(x,i) for x in userIds.index.values], index = userIds.index)
#userIds.index[userIds.loc[:,"0"] == userIds.loc[:,"1"]]

In [ ]:
if False:
    getProfileCounts(samplePlaytestUniqueProfilesPhase2, 2)

In [ ]:
if False:
    for userId in samplePlaytestUniqueProfiles[QUserId].unique():
        plotEventsPerMinute(userId, rmdfConcat, samplePlaytestUniqueProfiles)

In [ ]:
if False:
    for userId in samplePlaytestPretestPosttestUniqueProfiles[QUserId].unique():
        plotEventsPerMinute(userId, rmdfConcat, samplePlaytestUniqueProfiles)

------------

In [ ]:
gformProfiles = getUniqueProfiles(gform)
wholeProfiles = getUniqueProfiles(sampleWhole)
linuxProfiles = getUniqueProfiles(sampleLinux)
len(gformProfiles), len(wholeProfiles), len(linuxProfiles)

In [ ]:
playtestProfiles = getUniqueProfiles(samplePlaytest)
playtestProfilesPhase1 = getUniqueProfiles(samplePlaytestPhase1)
playtestProfilesPhase2 = getUniqueProfiles(samplePlaytestPhase2)

assert(len(playtestProfiles) == len(playtestProfilesPhase1) + len(playtestProfilesPhase2))

len(playtestProfiles),\
len(playtestProfilesPhase1),\
len(playtestProfilesPhase2)

In [ ]:
shouldBeEmpty = samplePlaytest[((samplePlaytest[QTimestamp] > playtest2018Phase1EndDate)&(samplePlaytest[QTimestamp] < playtest2018Phase2StartDate))]
assert(len(shouldBeEmpty) == 0)

In [ ]:
playtestProfiles = playtestProfiles.dropna(how='any')
playtestProfilesPhase1 = playtestProfilesPhase1.dropna(how='any')
playtestProfilesPhase2 = playtestProfilesPhase2.dropna(how='any')

assert(len(playtestProfiles) == len(playtestProfilesPhase1) + len(playtestProfilesPhase2))

len(playtestProfiles),\
len(playtestProfilesPhase1),\
len(playtestProfilesPhase2)

In [ ]:
# intersections based on userId and/or questions
s1 = pd.merge(playtestProfilesPhase1, playtestProfilesPhase2, how='inner', on=[QUserId])
s2 = pd.merge(playtestProfilesPhase1, playtestProfilesPhase2, how='inner', on=[QUserId, QAge, QGender])
len(s1), len(s2)

In [ ]:
playtestProfiles[QUserId].nunique(), playtestProfilesPhase1[QUserId].nunique(), playtestProfilesPhase2[QUserId].nunique(), 

#### RedMetrics dataframes

In [ ]:
# rmdfConcat is rmdf1522 + rmdf160
# rmdfLinuxPlatform

# sample by time slots
rmdfPlaytestPhase1 = rmdfLinuxPlatform[rmdfLinuxPlatform[QUserId].isin(samplePlaytestPhase1[QUserId])]
rmdfPlaytestPhase1 = rmdfPlaytestPhase1[rmdfPlaytestPhase1['userTime'].apply(isPlaytestPhase1)]
rmdfPlaytestPhase2 = rmdfLinuxPlatform[rmdfLinuxPlatform[QUserId].isin(samplePlaytestPhase2[QUserId])]
rmdfPlaytestPhase2 = rmdfPlaytestPhase2[rmdfPlaytestPhase2['userTime'].apply(isPlaytestPhase2)]
rmdfPlaytest = pd.concat([rmdfPlaytestPhase1, rmdfPlaytestPhase2])

# sample by filtering out userIds with multiple profiles
#  better solution: take those multiple profiles into account, eg by creating new userIds for them
rmdfPlaytestUniqueProfilesPhase1 = rmdfPlaytestPhase1[rmdfPlaytestPhase1[QUserId].isin(samplePlaytestUniqueProfilesPhase1[QUserId])]
rmdfPlaytestUniqueProfilesPhase2 = rmdfPlaytestPhase2[rmdfPlaytestPhase2[QUserId].isin(samplePlaytestUniqueProfilesPhase2[QUserId])]
rmdfPlaytestUniqueProfiles = pd.concat([rmdfPlaytestUniqueProfilesPhase1, rmdfPlaytestUniqueProfilesPhase2])

# sample by keeping only pretest-posttest pairs
rmdfPlaytestPretestPosttestUniqueProfilesPhase1 = rmdfPlaytestUniqueProfilesPhase1[rmdfPlaytestUniqueProfilesPhase1[QUserId]\
                                  .isin(samplePlaytestPretestPosttestUniqueProfilesPhase1[QUserId])]
rmdfPlaytestPretestPosttestUniqueProfilesPhase2 = rmdfPlaytestUniqueProfilesPhase2[rmdfPlaytestUniqueProfilesPhase2[QUserId]\
                                  .isin(samplePlaytestPretestPosttestUniqueProfilesPhase2[QUserId])]
rmdfPlaytestPretestPosttestUniqueProfiles = pd.concat(\
                                  [rmdfPlaytestPretestPosttestUniqueProfilesPhase1, rmdfPlaytestPretestPosttestUniqueProfilesPhase2])

# sample according to the answer to the Volunteering question
rmdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1 =\
                rmdfPlaytestPretestPosttestUniqueProfilesPhase1[rmdfPlaytestPretestPosttestUniqueProfilesPhase1[QUserId]\
                .isin(samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase1[QUserId])]
rmdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase2 =\
                rmdfPlaytestPretestPosttestUniqueProfilesPhase2[rmdfPlaytestPretestPosttestUniqueProfilesPhase2[QUserId]\
                .isin(samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase2[QUserId])]
rmdfPlaytestPretestPosttestUniqueProfilesVolunteers       =\
                rmdfPlaytestPretestPosttestUniqueProfiles[rmdfPlaytestPretestPosttestUniqueProfiles[QUserId]\
                .isin(samplePlaytestPretestPosttestUniqueProfilesVolunteers[QUserId])]

In [ ]:
assert(rmdfPlaytest[QUserId].nunique()       == samplePlaytest[QUserId].nunique())
assert(rmdfPlaytestPhase1[QUserId].nunique() == samplePlaytestPhase1[QUserId].nunique())
assert(rmdfPlaytestPhase2[QUserId].nunique() == samplePlaytestPhase2[QUserId].nunique())

### players

In [ ]:
thresholdGameEventsCount = 10
thresholdTimedelta = pd.Timedelta('0 days 00:05:00.000000')

playedEnoughTime = []
playedEnoughEvents = []

for userId in samplePlaytest[QUserId].unique():
    rows = rmdfPlaytest[rmdfPlaytest[QUserId] == userId]
    if len(rows) > thresholdGameEventsCount:
        playedEnoughEvents.append(userId)
    delta = rows['userTime'].max() - rows['userTime'].min()
    if delta > thresholdTimedelta:
        playedEnoughTime.append(userId)
        
len(samplePlaytest[QUserId].unique()), len(playedEnoughTime), len(playedEnoughEvents)

In [ ]:
setCount(samplePlaytestPhase1, phase1Label, playersCountLabel)
setCount(samplePlaytestPhase2, phase2Label, playersCountLabel)
setCount(samplePlaytest,       totalLabel,  playersCountLabel)
    
#assert(playtestStatistics.loc[totalLabel, playersCountLabel] == \
#    playtestStatistics.loc[phase1Label, playersCountLabel]\
#    + playtestStatistics.loc[phase2Label, playersCountLabel])

playtestStatistics

### all playtest respondents

In [ ]:
if displayResults:
    plotGender(samplePlaytest,'Sexe des participants')

In [ ]:
if displayResults:
    plotGender(samplePlaytestPhase1,'Gender of participants', labels = ['Female', 'Male', 'Other/Not given'])

In [ ]:
if displayResults:
    plotGender(samplePlaytestPhase2,'Sexe des participants, phase 2')

In [ ]:
if displayResults:
    plotAge(samplePlaytest, "Âge des participants, total")
    plotAge(samplePlaytestPhase1, "Age of participants", xlabel = 'Age', ylabel = 'Number')
    plotAge(samplePlaytestPhase2, "Âge des participants, phase 2")
    plotAge(samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase1, "Age of volunteers", xlabel = 'Age', ylabel = 'Number')

In [ ]:
if displayResults:
    plotAgeGroups(samplePlaytest, pivotalAgesList = [1,15,25,35,45], title = 'Âge des participants', showCount = False, unknownAgeLabel ="inconnu")
    plotAgeGroups(samplePlaytestPhase1, pivotalAgesList = [1,15,25,35,45], title = 'Age of participants', showCount = False)
    plotAgeGroups(samplePlaytestPhase2, pivotalAgesList = [1,15,25,35,45], title = 'Âge des participants, phase 2', showCount = False)
    plotAgeGroups(samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase1, pivotalAgesList = [1,15,25,35,45], title = 'Age of volunteers', showCount = False)

In [ ]:
setGenderedCount(samplePlaytestPhase1, phase1Label, respondentsCountLabel)
setGenderedCount(samplePlaytestPhase2, phase2Label, respondentsCountLabel)
setGenderedCount(samplePlaytest,       totalLabel,  respondentsCountLabel,\
                 plotPie = displayResults, title = 'Gender of respondents')

assert(playtestStatistics.loc[totalLabel, respondentsCountLabel] ==\
       playtestStatistics.loc[phase1Label, respondentsCountLabel]\
      + playtestStatistics.loc[phase2Label, respondentsCountLabel])

playtestStatistics

In [ ]:
setGenderedCount(samplePlaytestUniqueProfilesPhase1, phase1Label, respondentsExploitableCountLabel)
setGenderedCount(samplePlaytestUniqueProfilesPhase2, phase2Label, respondentsExploitableCountLabel)
setGenderedCount(samplePlaytestUniqueProfiles,       totalLabel,  respondentsExploitableCountLabel,\
                 plotPie = displayResults, title = 'Gender of exploitable respondents')

assert(playtestStatistics.loc[totalLabel, respondentsExploitableCountLabel] ==\
       playtestStatistics.loc[phase1Label, respondentsExploitableCountLabel]\
      + playtestStatistics.loc[phase2Label, respondentsExploitableCountLabel])

playtestStatistics

In [ ]:
# answers per userId
# => gform.indexes per userId
#indicesPerUserId = gform[QUserId].value_counts()
#indicesPerUserId

### twice respondents

In [ ]:
setGenderedCount(samplePlaytestPretestPosttestUniqueProfilesPhase1, phase1Label, respondentsTwiceCountLabel)
setGenderedCount(samplePlaytestPretestPosttestUniqueProfilesPhase2, phase2Label, respondentsTwiceCountLabel)
setGenderedCount(samplePlaytestPretestPosttestUniqueProfiles,       totalLabel,  respondentsTwiceCountLabel,\
                 plotPie = displayResults, title = 'Gender of participants (pretest-posttest)')

assert(playtestStatistics.loc[totalLabel, respondentsTwiceCountLabel] ==\
       playtestStatistics.loc[phase1Label, respondentsTwiceCountLabel]\
      + playtestStatistics.loc[phase2Label, respondentsTwiceCountLabel])

playtestStatistics

In [ ]:
if displayResults:
    (samplePlaytestPretestPosttestUniqueProfiles, 'Age repartition of pretest-posttest participants')

### Volunteers

In [ ]:
setGenderedCount(samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase1, phase1Label, respondentsTwiceVolunteersCountLabel,\
                 plotPie = displayResults, title = 'Gender of participants (pretest-posttest volunteers)')
setGenderedCount(samplePlaytestPretestPosttestUniqueProfilesVolunteersPhase2, phase2Label, respondentsTwiceVolunteersCountLabel)
setGenderedCount(samplePlaytestPretestPosttestUniqueProfilesVolunteers,       totalLabel,  respondentsTwiceVolunteersCountLabel,\
                 plotPie = displayResults, title = 'Gender of participants (pretest-posttest volunteers)')

assert(playtestStatistics.loc[totalLabel, respondentsTwiceVolunteersCountLabel] ==\
       playtestStatistics.loc[phase1Label, respondentsTwiceVolunteersCountLabel]\
      + playtestStatistics.loc[phase2Label, respondentsTwiceVolunteersCountLabel])

playtestStatistics

### Sankey diagram

In [ ]:
from matplotlib.sankey import Sankey

In [ ]:
groupLabel = phase1Label
players = playtestStatistics.loc[groupLabel, playersCountLabel]
minus = pd.Series()

minus[respondentsCountLabel]                = playtestStatistics.loc[groupLabel, respondentsCountLabel]\
												- playtestStatistics.loc[groupLabel, playersCountLabel]
minus[respondentsExploitableCountLabel]     = playtestStatistics.loc[groupLabel, respondentsExploitableCountLabel]\
												- playtestStatistics.loc[groupLabel, respondentsCountLabel]
minus[respondentsTwiceCountLabel]           = playtestStatistics.loc[groupLabel, respondentsTwiceCountLabel]\
												- playtestStatistics.loc[groupLabel, respondentsExploitableCountLabel]
minus[respondentsTwiceVolunteersCountLabel] = playtestStatistics.loc[groupLabel, respondentsTwiceVolunteersCountLabel]\
												- playtestStatistics.loc[groupLabel, respondentsTwiceCountLabel]

In [ ]:
if displayResults:
#if True: 
    # source: https://flothesof.github.io/sankey-tutorial-matplotlib.html
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[])
    diagrams = Sankey(flows=[
               #input
                        players,
               #output
                        -players-minus.sum(),
               #losses
                        minus[respondentsTwiceVolunteersCountLabel], 
                        minus[respondentsTwiceCountLabel], 
                        minus[respondentsExploitableCountLabel], 
                        minus[respondentsCountLabel] 
                ],
           labels=[
               #input
                       playersCountLabel,
               #output
                       "exploitable sample",
               #losses
                       "non-respondents to optional questions",
                       "non-"+respondentsTwiceCountLabel,
                       "technical error",
                       "non-"+respondentsCountLabel
                ],

           orientations=[
               #input
                        0,
               #output
                        0,
               #losses
                        -1, -1, -1, -1
               ],
           rotation=-90,
            ax=ax,
           scale = 1/players
          ).finish()
    plt.title("Playtest participants flow")
    for diagram in diagrams:
        diagram.text.set_fontweight('bold')
        diagram.text.set_fontsize('20')
        for text in diagram.texts:
            text.set_fontsize('14')

#### drafts

In [ ]:
gform[gform[QTemporality] == answerTemporalities[0]][QEnjoyed].apply(str).value_counts()

In [ ]:
gform[QAge].apply(str).value_counts()['nan']

In [ ]:
gform[gform[QAge].apply(str) == 'nan'][QTemporality].unique()